In [ ]:
import os
import torch
import logging
import pandas as pd
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import nn, optim

# --- Config ---
base_data_dir = "/your/path/to/train_folders"
save_dir = "/your/path/to/save_weights"
log_file = os.path.join(save_dir, "training_log.csv")
log_txt = os.path.join(save_dir, "training_hpc_log.txt")
image_folders = [
    "train",
    "train_bright_50",
    "train_bright_-50",
    "train_contrast_50",
    "train_contrast_-50"
]
epochs = 2
batch_size = 64
lr = 1e-4
layer_indices_to_save = [0, 2, 5, 7]

# --- Setup Logging ---
os.makedirs(save_dir, exist_ok=True)
logging.basicConfig(
    filename=log_txt,
    level=logging.INFO,
    format='%(asctime)s %(levelname)s: %(message)s',
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Transforms ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Train and Save ---
def train_on_folder(folder_name):
    data_path = os.path.join(base_data_dir, folder_name)

    # Load all images in the folder
    train_dataset = datasets.ImageFolder(data_path, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    # Load and partially unfreeze VGG16
    vgg16 = models.vgg16(pretrained=True)
    for param in vgg16.parameters():
        param.requires_grad = False
    for i in range(8):  # conv1_1 to conv2_2
        if isinstance(vgg16.features[i], nn.Conv2d):
            for param in vgg16.features[i].parameters():
                param.requires_grad = True
    vgg16.classifier[6] = nn.Linear(4096, 200)
    vgg16 = vgg16.to(device)

    optimizer = optim.Adam(filter(lambda p: p.requires_grad, vgg16.parameters()), lr=lr)
    criterion = nn.CrossEntropyLoss()

    loss_records = []
    vgg16.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = vgg16(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        loss_records.append({
            "condition": folder_name,
            "epoch": epoch + 1,
            "loss": total_loss
        })

    # Save weights from specified conv layers
    for idx in layer_indices_to_save:
        layer = vgg16.features[idx]
        weights = layer.weight.detach().cpu()
        save_path = os.path.join(save_dir, f"{folder_name}_layer_{idx}.pt")
        torch.save(weights, save_path)
        logging.info(f"Saved weights: {save_path}")

    return loss_records

# --- Main ---
if __name__ == "__main__":
    all_logs = []
    for folder in image_folders:
        logging.info(f"Started training on: {folder}")
        logs = train_on_folder(folder)
        all_logs.extend(logs)
        logging.info(f"Finished training on: {folder}")

    # Save training log
    df = pd.DataFrame(all_logs)
    df.to_csv(log_file, index=False)
    logging.info(f"Training log saved to: {log_file}")